In [8]:
# %load Train_people.py
import argparse
#import cv2
import numpy as np
import torch
from torch.autograd import Function
from torchvision import models, transforms
from model import *
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader,random_split
#from torch.utils.tensorboard import SummaryWriter
from _datetime import date
import torchvision
import torch.nn as nn
from CIFAR_dataset import *


def get_args():
    parser = argparse.ArgumentParser()
    parser.add_argument('--batch',type=int, default=5)
    parser.add_argument('--epoch', type=int , default=10)
    args = parser.parse_args()

    return args


"""
Get CIFAR-100 Dataset 
Get VGG16 Network 
Train on coarse labels 
Save weights 
"""
#args =get_args()
epochs = 10
batch_size =32
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print("Using Device: ", device)


#writer = SummaryWriter("runs_2/")

transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.507, 0.487, 0.441], std=[0.267, 0.256, 0.276])
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.507, 0.487, 0.441], std=[0.267, 0.256, 0.276])
])

dataset = CIFAR100(root='people_class/', train=True, download=True, coarse=True, transform=transform_train)
test_dataset = CIFAR100(root='people_class/', train=False, download=True, coarse=True, transform=transform_test)

#lengths = [int(len(dataset)*0.8), int(len(dataset)*0.2)]

train_loader = DataLoader(dataset, batch_size=batch_size,shuffle=True,num_workers=4)
test_loader = DataLoader(test_dataset,batch_size=batch_size,shuffle=False,num_workers=1)
model = models.vgg16(pretrained=True)
model.classifier[6] = nn.Linear(4096,20)
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

print("My_model", model)
prev_acc = 0

for epoch in range(1,epochs+1):
    model.train()
    epoch_loss = 0.0

    for i,data in enumerate(train_loader):
        optimizer.zero_grad()
        label = data[2].to(device)
        #img = torch.squeeze(data[0]).permute(2,1,0)
        #img = data[0]
        output = model(data[0].to(device))
        #print(output.shape)
        loss = criterion(output,label)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
        if(i%100==0):
            print('[%d, %5d] loss: %.3f' %(epoch, i + 1, epoch_loss / 100))
            writer.add_scalar("Training Loss",epoch_loss/100, epoch-1)
            epoch_loss = 0.0


    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for i,data in enumerate(test_loader):
            img, label = data[0], data[2]
            img = img.to(device)
            label = label.to(device)
            test_out = model(img)
            _,predicted = torch.max(test_out.data,1)
            #print("Predicted Value: ", predicted)
            total += label.size(0)
            correct += (predicted==label).sum().item()
            #print("Total" , total)
            #print("Correct: ", correct)

        current_acc = (100 * correct / total)
        print('Current Test Accuracy: %d %%' %(current_acc) )
        writer.add_scalar("Testing Accuracy", (current_acc), epoch - 1)
        if(current_acc>=prev_acc):
            print("Saving Weights")
            torch.save(model.state_dict(),"checkpoints/best_cifar.pt")

print('Training Model on Cifar compleleted')







Using Device:  cpu
Files already downloaded and verified
My_model VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)


RuntimeError: size mismatch, m1: [32 x 512], m2: [25088 x 4096] at /Users/distiller/project/conda/conda-bld/pytorch_1579022061893/work/aten/src/TH/generic/THTensorMath.cpp:136